<a href="https://colab.research.google.com/github/Sababagherinia/sentimental-analysis/blob/main/nlp_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#seniment analysis on sentipers dataset

## installing necessary packages

In [ ]:
!pip install hazm

In [ ]:
! pip install keras

In [ ]:
!pip install https://github.com/sobhe/hazm/archive/master.zip --upgrade

  Using cached https://github.com/sobhe/hazm/archive/master.zip
  Created wheel for hazm: filename=hazm-0.7.1-cp37-none-any.whl size=318809 sha256=7a063a29392f9f9bea70e0e4a2f1035cebcab7fef1a6195229290faa84a03974
  Stored in directory: /tmp/pip-ephem-wheel-cache-z7vlq98h/wheels/32/97/cc/640e3b2f3e7b53c5ae336824a31184e86072b8ded31dc0151e
Successfully built hazm
  Found existing installation: hazm 0.7.1
    Uninstalling hazm-0.7.1:
      Successfully uninstalled hazm-0.7.1


## importing general moduels

In [ ]:
import pandas as pd
import re

import math
import numpy as np
from hazm import *
from sklearn.feature_extraction.text import TfidfVectorizer

##loading dataset

In [ ]:
dataset = pd.read_excel('/content/sample_data/sentipers.xlsx')
print(dataset.head(10))
print(dataset.shape)
print(dataset.info())
print(dataset.describe())

   index     sid  ... polarity                             file
0      0   rev-1  ...        0  data/main/HP LaserJet M1132.xml
1      1   rev-2  ...        0  data/main/HP LaserJet M1132.xml
2      2   rev-3  ...        0  data/main/HP LaserJet M1132.xml
3      3   rev-4  ...        0  data/main/HP LaserJet M1132.xml
4      4   rev-5  ...        2  data/main/HP LaserJet M1132.xml
5      5   rev-6  ...        0  data/main/HP LaserJet M1132.xml
6      6   rev-7  ...        1  data/main/HP LaserJet M1132.xml
7      8   rev-9  ...        0  data/main/HP LaserJet M1132.xml
8      9  rev-10  ...        1  data/main/HP LaserJet M1132.xml
9     10  rev-11  ...        0  data/main/HP LaserJet M1132.xml

[10 rows x 5 columns]
(15683, 5)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15683 entries, 0 to 15682
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   index     15683 non-null  int64 
 1   sid       15683 non-null  object
 2

##preprocessing

### removing irrelevant coulmns

In [ ]:
dataset = dataset[['text', 'polarity']]
print(dataset.head(10))

                                                text  polarity
0  اینک قصد داریم پرینتر دیگری از پرینترهای لیزری...         0
1  پرینتری چند کاره از رده‌ی Entry Level یا سطح م...         0
2  به هر صورت اکنون ما در دنیایی زندگی می‌کنیم،  ...         0
3  به صورتی که توانایی کپی کردن،  اسکن،  فکس،  پر...         0
4  به هر صورت معمولا چیزی که بیشتر کاربران از پری...         2
5     همچنین عمل اسکن را هم باید خیلی خوب انجام دهد.         0
6  M1132 نیز پرینتری است که به نظر یکی از بهترین‌...         1
7  همانطور که گفتیم،  M1132 پرینتری لیزری،  تک رن...         0
8  ابعاد آن 415 x 265 x 250 میلی متر می‌باشد و وز...         1
9  این پرینتر بدنه‌ی مشکی استایلیش و البته نه خیل...         0


###seperating query and target set

In [ ]:
X = dataset.iloc[:, 0]
Target = dataset.iloc[:, -1]
print(X.head(10))
print(X.shape)
print(Target.head(10))
print(Target.shape)

0    اینک قصد داریم پرینتر دیگری از پرینترهای لیزری...
1    پرینتری چند کاره از رده‌ی Entry Level یا سطح م...
2    به هر صورت اکنون ما در دنیایی زندگی می‌کنیم،  ...
3    به صورتی که توانایی کپی کردن،  اسکن،  فکس،  پر...
4    به هر صورت معمولا چیزی که بیشتر کاربران از پری...
5       همچنین عمل اسکن را هم باید خیلی خوب انجام دهد.
6    M1132 نیز پرینتری است که به نظر یکی از بهترین‌...
7    همانطور که گفتیم،  M1132 پرینتری لیزری،  تک رن...
8    ابعاد آن 415 x 265 x 250 میلی متر می‌باشد و وز...
9    این پرینتر بدنه‌ی مشکی استایلیش و البته نه خیل...
Name: text, dtype: object
(15683,)
0    0
1    0
2    0
3    0
4    2
5    0
6    1
7    0
8    1
9    0
Name: polarity, dtype: int64
(15683,)


### clearing our texts

some actoins like normalizing texts, stemming,removing stop words and lemmatization

In [ ]:
normalizer = Normalizer()
stemmer = Stemmer()
lemmatizer = Lemmatizer()
def preprocessing(text):
    cleared_text = []
    text = re.sub('[۰۱۲۳۴۵۶۷۸۹]', '', text)
    text = re.sub('[0-9]', '', text)
    text = re.sub('[$!?/,.%+*_^():{}"؟><]', '', text)
    text = re.sub("-", '', text)
    text = re.sub("'", '', text)
    text = normalizer.normalize(text)
    text = text.split()
    for w in text:
          if len(w) > 2:
              w = stemmer.stem(w)
              w = lemmatizer.lemmatize(w)
              cleared_text.append(w)
    cleared_text = ' '.join(cleared_text)
    return cleared_text
clean_docs = []
for doc in X.values:
    clean_docs.append(preprocessing(doc))
print(clean_docs[1])

پرینتر چند کاره رده Entry Level سطح مبتد


###vectorization

####vectorizing text by tf-idf method

In [ ]:
vectorizer = TfidfVectorizer()
X_vector = vectorizer.fit_transform(clean_docs)
feature_names = vectorizer.get_feature_names
print(X_vector.shape)

(15683, 9214)


####spliting data set to train set and test set

using sklearn split method to split our data by 2 to 8 ratio and 0 random state

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Target_train, Target_test = train_test_split(X_vector, Target, test_size=0.2, random_state=0)
print("X_train", X_train.shape)
print("X_test:", X_test.shape)

X_train (12546, 9214)
X_test: (3137, 9214)


In [ ]:
print(X_vector)
print(len(vectorizer.get_feature_names()))

  (0, 8826)	0.15192382756788228
  (0, 6674)	0.31528301569964334
  (0, 8815)	0.29787474645091455
  (0, 6206)	0.40287848167797413
  (0, 8299)	0.3662100190192315
  (0, 4612)	0.22793932868900824
  (0, 8297)	0.31963942996245065
  (0, 4224)	0.1639387396131728
  (0, 6043)	0.3423919382068705
  (0, 2561)	0.44060379546835665
  (1, 6280)	0.34623342793433687
  (1, 5090)	0.2750071092106493
  (1, 734)	0.4615781705879867
  (1, 387)	0.4615781705879867
  (1, 4695)	0.2791110150880839
  (1, 8644)	0.35553141014246564
  (1, 8517)	0.23734555400816648
  (1, 8299)	0.3431417109430171
  (2, 4251)	0.13931095905375118
  (2, 4126)	0.16247081247947787
  (2, 4406)	0.19155614895689255
  (2, 4965)	0.25877078573468126
  (2, 9001)	0.2521256481251571
  (2, 5684)	0.2518023336116107
  (2, 2704)	0.16139416863289016
  :	:
  (15681, 4251)	0.21490309187266077
  (15681, 4224)	0.20356077627743255
  (15682, 761)	0.2169014421393742
  (15682, 7746)	0.24561041819004018
  (15682, 7831)	0.2066257325986272
  (15682, 6405)	0.20781377435

converting our sparse matrices to dense matrices

In [ ]:
X_train_dense = X_train.toarray()
X_test_dense = X_test.toarray()
# print(X_train.shape)
# X_list = X_list.tolist()
# X_df = pd.DataFrame(X_train, columns=feature_names)
# print(X_df.head(10))

##learning Classifiers

###naive bayes classifier

using sklearn library to learn and predict naive bayes classifier

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.naive_bayes import GaussianNB

classifier = GaussianNB()
classifier.fit(X_train_dense, Target_train)

Target_predict = classifier.predict(X_test_dense)

printing evaluation metrics and confusion matrix for naive bayes

In [ ]:
print(classification_report(Target_test, Target_predict))
print(confusion_matrix(Target_test, Target_predict))

print('accuracy of NB is:', accuracy_score(Target_test, Target_predict))

              precision    recall  f1-score   support

          -2       0.02      0.19      0.04        37
          -1       0.15      0.52      0.24       318
           0       0.62      0.25      0.36      1182
           1       0.42      0.23      0.29      1000
           2       0.32      0.37      0.34       600

    accuracy                           0.29      3137
   macro avg       0.31      0.31      0.25      3137
weighted avg       0.44      0.29      0.32      3137

[[  7  19   6   4   1]
 [ 44 165  37  38  34]
 [116 376 297 190 203]
 [120 314 101 226 239]
 [ 58 200  36  84 222]]
accuracy of NB is: 0.29231750079693974


###Logistic regression

using sklearn library to learn and predict logistic regression classifier

In [ ]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression()
classifier.fit(X_train_dense, Target_train)

Target_predict = classifier.predict(X_test_dense)

print(classification_report(Target_test, Target_predict))
print(confusion_matrix(Target_test, Target_predict))
print('Accuracy of regression classifier is:', accuracy_score(Target_test, Target_predict))

### Neural Network classifier

#### restart the session before fitting NN model again

In [ ]:
from keras.backend import clear_session
clear_session()

splititg our training set to train and validation set by 25 to 75 ratio and random state 0

In [ ]:
X_trainN, X_eval, Target_trainN, Target_eval = train_test_split(X_train_dense, Target_train, test_size=0.25, random_state=0)

In [ ]:
input_dim = X_trainN.shape[1]
print(input_dim)

9214


importing keras moduels and creating our network layers

In [ ]:
from keras.models import Sequential
from keras import layers

model = Sequential()
model.add(layers.Dense(10, input_dim=input_dim, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

compiling model and printing a summary of it's structure

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 10)                92150     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 11        
Total params: 92,161
Trainable params: 92,161
Non-trainable params: 0
_________________________________________________________________


learning model with training set and validation set

In [ ]:
model.fit(X_trainN, Target_trainN, epochs=100, verbose=False, validation_data=(X_eval, Target_eval))

evaluate learned model on test set and printing accuarcy

In [ ]:
loss, accuracy = model.evaluate(X_test_dense, Target_test, verbose=False)
print("Testing Accuracy: ", accuracy)
print("Testing Loss: ", loss)

Testing Accuracy:  0.48868346214294434
Testing Loss:  -242.3919677734375


####printing confusion matrix and evaluation metrics of NN

In [ ]:
predicted = model.predict_classes(X_test_dense)
print(classification_report(Target_test, predicted))
print(confusion_matrix(Target_test, predicted))
print('Accuracy of NN classifier is:', accuracy_score(Target_test, predicted))

/usr/local/lib/python3.7/dist-packages/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


              precision    recall  f1-score   support

          -2       0.00      0.00      0.00        37
          -1       0.00      0.00      0.00       318
           0       0.59      0.66      0.62      1182
           1       0.42      0.76      0.54      1000
           2       0.00      0.00      0.00       600

    accuracy                           0.49      3137
   macro avg       0.20      0.28      0.23      3137
weighted avg       0.35      0.49      0.40      3137

[[  0   0  27  10   0]
 [  0   0 244  74   0]
 [  0   0 777 405   0]
 [  0   0 244 756   0]
 [  0   0  33 567   0]]
Accuracy of NN classifier is: 0.48868345553076187


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
